# Import necessary libraries

In [1]:
import stmpy
import stmpy.tools as st
import stmpy.driftcorr as dfc

%pylab inline

# pylab.style.use('Thin')

Populating the interactive namespace from numpy and matplotlib


# Loop through all the topos in the same folder

In [ ]:
# Load all the files ending with ".sxm"

import os

d = []
files = []
for ix in os.listdir():
    if ix.endswith('.sxm'):
        files.append(ix[:-4])
        d.append(stmpy.load(ix))

In [ ]:
# automatically create useful attributes to store information of the topos

for ix in d:
    try:
        ix.z = st.lineSubtract(ix.Z, 2, maskon=True)
        ix.x = float(ix.header['scan_range'][0])
        ix.y = float(ix.header['scan_range'][1])
        ix.bias = float(ix.header['bias'])*1e3
        ix.current = float(ix.header['current>current (a)'])*1e12
    except TypeError:
        ix.z = st.lineSubtract(ix.Z, 2, maskon=False)
        ix.x = float(ix.header['scan_range'][0])
        ix.y = float(ix.header['scan_range'][1])
        ix.bias = float(ix.header['bias'])*1e3
        ix.current = float(ix.header['current>current (a)'])*1e12

In [ ]:
# Display all the topos with index labelled

for i, ix in enumerate(d):
    c = mean(ix.z)
    s = std(ix.z)
    figure()
    imshow(ix.z, extent=[0, ix.x, 0, ix.y], cmap=stmpy.cm.blue2, clim=[c-3*s, c+3*s])
    gca().axes.get_xaxis().set_visible(False)
    gca().axes.get_yaxis().set_visible(False)
    gca().set_frame_on(False)
    gca().set_aspect(1)
    stmpy.image.add_label('{}'.format(i), ax=gca())
#     savefig('{} at {} mV {} pA.png'.format(files[i], int(ix.bias), int(ix.current)), 
#             dpi=400, bbox_inches='tight', pad_inches=0)

# Drift correct and take linecut for a 3ds file

In [ ]:
# Load the data
d = stmpy.load('Grid Spectroscopy001.3ds')

In [ ]:
# Remove background from topo

z = st.lineSubtract(d.Z, 1)

# If the topo is too cappy, the averaged LIY can be used for drift correction
# z = mean(d.LIY, axis=0)

## Drift correct the dos map

In [ ]:
# Find the dominant Bragg peaks

bp1 = dfc.findBraggs(z, r=0.5, w=0.01, show=True)

In [ ]:
# Optional: if your piezo calibration is off, you can generate correct Bragg peaks first

# bpc = dfc.generate_bp(t.z, bp=bp1, angle=pi/2, orient=pi/4)
# bpc

In [ ]:
# Find the drifting field from topo
z_c, p1 = dfc.find_drift_parameter(z, bp_c=bp1, sigma=10, show=True)

In [ ]:
# Apply the drifting field to LIY layers

d.liy_c = dfc.apply_drift_parameter(d.LIY, p=p1)

## Take linecuts

In [ ]:
# take linecut -- also works for non-square dataset

to_cut = d.fft_c
# Here are the angles for the linecuts
angles = []
offset = 82 # first angle of linecut in the unit of degrees
width = 7 # width to average the linecut

to_plot = np.mean(to_cut, axis=0)
H, W = np.shape(to_plot)

# This is the length of the linecut
L = W // 2 * 1
center = (np.array(np.shape(to_plot))[::-1]-1) // 2

for i in range(4):
    angles.append(offset/180*np.pi+i*np.pi/4)

colors = stmpy.cm.rainbow(np.linspace(0, 1, len(angles)))

c = np.mean(to_plot)
s = np.std(to_plot)

figure(figsize=[4,4])
imshow(to_plot, clim=[0, c+3*s], cmap=stmpy.cm.gray_r)

rs = []
cuts = []

for i in range(len(angles)):
    p0 = center + np.array([L * cos(angles[i]), L * sin(angles[i]) * H / W])
    p1 = center - np.array([L * cos(angles[i]), L * sin(angles[i]) * H / W])
    
    r, cut = stmpy.tools.linecut(d.fft_c, p0=p0, p1=p1, width=width, show=True, ax=gca(), color=colors[i])
    rs.append(r)
    cuts.append(cut)

In [ ]:
# Calibrate the qscale of the linecut -- to be added

In [ ]:
# Display the linecuts

to_plot = cuts[0]
qscale = 1.2
thres = 3 # threshold for color limit
en0 = np.min(d.en)
en1 = np.max(d.en)

extents = [-qscale, qscale, en0, en1]

# # normalize by energy 
# for i in range(len(to_plot)):
#     to_plot[i] /= mean(to_plot[i])

# # normalize by momentum:
# for i in range(shape(to_plot)[-1]):
#     to_plot[:, i] /= mean(to_plot[:, i])

c = mean(to_plot)
s = std(to_plot)
figure()
imshow(to_plot, extent=extents, clim=[0, c+thres*s], cmap=stmpy.cm.gray_r)
# savefig('blue.png', dpi=400, bbox_inches='tight', pad_inches=0)